In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2598)

In [15]:
x[:,:2]

matrix([[  644016.20653445,   631406.3152466 ],
        [  674136.40891449,   673679.56986029],
        [ 1196545.02177413,  1201159.12541113],
        [ 1220596.81688074,  1256925.78210511],
        [  316074.22927967,   313204.37222633],
        [  398413.70457977,   388936.63386239],
        [  773680.00677557,   787606.07406631],
        [  783196.11645375,   757406.26361917],
        [  732985.20600945,   737035.73143686],
        [  762150.17758647,   757999.6253519 ],
        [  775551.76308323,   777052.72271506],
        [  669007.37515397,   688568.50058395],
        [  184198.28602509,   183485.61686012],
        [  194120.28284627,   189895.2240814 ],
        [  295975.7412995 ,   307763.42351634],
        [  269010.79847377,   257697.6743093 ],
        [  400755.09338018,   402614.90400025],
        [  404714.37530819,   398862.65285996],
        [  659119.79227517,   634879.0084836 ],
        [  613854.98378807,   609965.43590426],
        [ 1029833.77475012,  1048104.709

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-02, 5e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.48180968e+07  0.00000000e+00  6.11e+03 3.01e+03  1.00e+06     0s
   1  -5.66598855e+07 -1.66521320e+07  2.08e+03 1.03e+03  3.86e+05     0s
   2  -4.38930819e+07 -4.29010381e+07  1.44e+02 7.11e+01  7.93e+04     0s
   3  -4.29635306e+07 -4.32007561e+07  1.43e-01 7.07e-02  4.3

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[8.290731958846943e-10,
 442611.11887399777,
 137862.88621161348,
 4.521761220446438e-06,
 86182.83843418928,
 6.9336881093609234e-06,
 1.8717093761798365e-05,
 2.6019939645567704e-09,
 5.114329595889112e-11,
 598751.2700874457,
 5.114259853398521e-11,
 5.114328429901276e-11,
 5.114293501631473e-11,
 5.148737849370587e-11,
 386764.90515731205,
 8.884345569612342e-11,
 1.0011761614500914e-08,
 5.599689723106115e-06,
 117467.56752598129,
 980837.2807496139,
 4.345275558615205e-06,
 282910.89091243607,
 409078.7491175158,
 2.0959526122480354e-08,
 94370.44698601533,
 30767.342471865577,
 1.3616134053036182e-06,
 341141.7729942375,
 587604.0231315054,
 8.884344665300115e-11,
 1.9566700964869315e-08,
 147622.6466998118,
 200818.79378562974,
 1.564056254270489e-08,
 39870.427057182074,
 477614.09873524465,
 8.88434641909242e-11,
 1.9569787272378245e-08,
 6.416132121170315e-06,
 259938.6033094376,
 467411.5022767655,
 77050.58946412636,
 9.261653489238408e-07,
 8.883494703765562e-11,
 2.09786